In [1]:
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mplfinance as mpf
from datetime import datetime, timedelta
import requests
import os

# 在 Google Colab 上安装中文字体
!wget -O SimHei.ttf https://github.com/adobe-fonts/source-han-sans/raw/release/TTF/SourceHanSansSC-Regular.ttf
import matplotlib
matplotlib.font_manager.fontManager.addfont('SimHei.ttf')
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置字体为思源黑体
plt.rcParams['axes.unicode_minus'] = False    # 解决负号显示问题

def download_stock_data(ticker, period='1y'):
    """
    使用 yfinance 下载指定股票代码和周期的历史数据。

    参数:
    ticker (str): 股票代码
    period (str): 数据周期，例如 '1y' 表示一年，'6mo' 表示六个月

    返回:
    data (DataFrame): 股票历史数据
    """
    data = yf.download(ticker, period=period)
    if data.empty:
        raise ValueError(f"无法下载股票数据，请检查股票代码和周期是否正确。")
    return data

def plot_candlestick_with_patterns(data, ticker, patterns):
    """
    绘制带有形态标记的 K 线图。

    参数:
    data (DataFrame): 股票历史数据
    ticker (str): 股票代码
    patterns (dict): 包含形态名称和对应日期列表的字典
    """
    # 将日期作为索引
    data.index.name = 'Date'

    # 创建一个副本，防止修改原始数据
    plot_data = data.copy()

    # 设置形态标记
    mc = mpf.make_marketcolors(up='r', down='g', edge='i', wick='i', volume='in')
    s = mpf.make_mpf_style(marketcolors=mc)

    # 添加形态标记
    apds = []
    for pattern_name, dates in patterns.items():
        for date in dates:
            # 如果日期在数据范围内
            if date in plot_data.index:
                idx = plot_data.index.get_loc(date)
                # 添加注释
                apds.append(mpf.make_addplot(
                    [plot_data['High'][idx] + plot_data['High'].max()*0.02 if i == idx else np.nan for i in range(len(plot_data))],
                    scatter=True, markersize=100, marker='v', color='blue',
                    label=pattern_name
                ))

    # 绘制图表
    fig, axlist = mpf.plot(plot_data, type='candle', style=s, volume=True,
                           addplot=apds, title=f"{ticker} K线图及形态标记",
                           returnfig=True, figsize=(14, 7))

    # 显示图例
    handles, labels = axlist[0].get_legend_handles_labels()
    if handles:
        axlist[0].legend(handles, labels)

    plt.show()

def detect_double_bottom(data):
    """
    检测双底形态（W形态）。

    参数:
    data (DataFrame): 股票历史数据

    返回:
    patterns (list): 包含双底形态的日期列表
    """
    patterns = []
    closes = data['Close'].values
    for i in range(1, len(closes)-1):
        # 检查是否形成双底形态
        if closes[i-1] > closes[i] < closes[i+1]:
            # 检查两个底部的价格差异是否在一定范围内
            if abs(closes[i] - closes[i-1]) / closes[i-1] < 0.03:
                patterns.append(data.index[i])
    return patterns

def detect_head_and_shoulders_bottom(data):
    """
    检测头肩底形态。

    参数:
    data (DataFrame): 股票历史数据

    返回:
    patterns (list): 包含头肩底形态的日期列表
    """
    patterns = []
    closes = data['Close'].values
    for i in range(3, len(closes)-3):
        left_shoulder = closes[i-3]
        head = closes[i-1]
        right_shoulder = closes[i+1]
        if left_shoulder > head < right_shoulder and left_shoulder > right_shoulder:
            patterns.append(data.index[i])
    return patterns

def detect_triple_bottom(data):
    """
    检测三重底形态。

    参数:
    data (DataFrame): 股票历史数据

    返回:
    patterns (list): 包含三重底形态的日期列表
    """
    patterns = []
    closes = data['Close'].values
    for i in range(2, len(closes)-2):
        if (closes[i-2] > closes[i-1] < closes[i] > closes[i+1] < closes[i+2]):
            patterns.append(data.index[i])
    return patterns

def detect_rounding_bottom(data):
    """
    检测圆弧底形态。

    参数:
    data (DataFrame): 股票历史数据

    返回:
    patterns (list): 包含圆弧底形态的日期列表
    """
    patterns = []
    closes = data['Close'].rolling(window=5).mean().values
    for i in range(5, len(closes)-5):
        if (closes[i-5] > closes[i] < closes[i+5]):
            patterns.append(data.index[i])
    return patterns

def detect_cup_and_handle(data):
    """
    检测杯柄形态。

    参数:
    data (DataFrame): 股票历史数据

    返回:
    patterns (list): 包含杯柄形态的日期列表
    """
    patterns = []
    closes = data['Close'].values
    for i in range(10, len(closes)-10):
        cup = closes[i-10:i]
        handle = closes[i:i+5]
        if (min(cup) == cup[-1] and max(handle) < max(cup)):
            patterns.append(data.index[i])
    return patterns

def detect_patterns(data):
    """
    识别各种技术形态。

    参数:
    data (DataFrame): 股票历史数据

    返回:
    patterns (dict): 包含形态名称和对应日期列表的字典
    """
    patterns = {}

    # 检测双底形态
    double_bottoms = detect_double_bottom(data)
    if double_bottoms:
        print(f"检测到 {len(double_bottoms)} 个双底形态")
        patterns['双底形态'] = double_bottoms
    else:
        print("未检测到双底形态")

    # 检测头肩底形态
    head_and_shoulders = detect_head_and_shoulders_bottom(data)
    if head_and_shoulders:
        print(f"检测到 {len(head_and_shoulders)} 个头肩底形态")
        patterns['头肩底形态'] = head_and_shoulders
    else:
        print("未检测到头肩底形态")

    # 检测三重底形态
    triple_bottoms = detect_triple_bottom(data)
    if triple_bottoms:
        print(f"检测到 {len(triple_bottoms)} 个三重底形态")
        patterns['三重底形态'] = triple_bottoms
    else:
        print("未检测到三重底形态")

    # 检测圆弧底形态
    rounding_bottoms = detect_rounding_bottom(data)
    if rounding_bottoms:
        print(f"检测到 {len(rounding_bottoms)} 个圆弧底形态")
        patterns['圆弧底形态'] = rounding_bottoms
    else:
        print("未检测到圆弧底形态")

    # 检测杯柄形态
    cup_and_handles = detect_cup_and_handle(data)
    if cup_and_handles:
        print(f"检测到 {len(cup_and_handles)} 个杯柄形态")
        patterns['杯柄形态'] = cup_and_handles
    else:
        print("未检测到杯柄形态")

    return patterns

def main():
    # 用户输入股票代码和周期
    ticker = input("请输入股票代码（例如 'AAPL'）: ").upper()
    period = input("请输入数据周期（例如 '1y' 表示一年，'6mo' 表示六个月，默认为 '1y'）: ")
    if not period:
        period = '1y'

    # 下载股票数据
    data = download_stock_data(ticker, period)

    # 识别形态
    patterns = detect_patterns(data)

    # 绘制带有形态标记的 K 线图
    plot_candlestick_with_patterns(data, ticker, patterns)

if __name__ == "__main__":
    main()


ModuleNotFoundError: No module named 'mplfinance'

In [ ]:
pip install mplfinance